# Linear Algebra Project - Markov Chains
This is the work I did for question 1d.

First we must initialize our transition matrix. The values are given by the weights / sum of weights from that edge.

In [ ]:
import numpy as np
import random

P = np.array([
    [0.0, 0.75, 0.25, 0.0, 0.0, 0.0],        # A
    [3/8, 0.0, 2/8, 0.0, 3/8, 0.0],          # B
    [0.0, 2/6, 0.0, 4/6, 0.0, 0.0],          # C
    [0.0, 0.0, 4/5, 0.0, 0.0, 1/5],          # D
    [0.0, 3/6, 2/6, 0.0, 0.0, 1/6],          # E
    [0.0, 0.0, 0.0, .5, .5, 0.0]           # F
])

Once initialized we can start making our functions.

The first function will be finding the real steady state in order to compare it to our results.
The steady state is determined by the equation $\pi P = \pi$.
We can rearrange this to make it into $\pi P - \pi = 0$.
This can be turned into $\pi(A - I) = 0$.
We do notice that this is in the form of xA=0 which is not what we want to solve as we need Ax=b. So we will transpose it which then gives us $A^T \pi^T = 0$.
This will work and allow us to use numpy to solve it.
We set the last row to be all 1s and the last element of b to be 1. This is because we know that the sum of the steady state vector must equal 1.

In [ ]:
def getSteadyState():
    n = len(P)
    A = np.transpose(P) - np.eye(n)
    A[-1] = np.ones(n)
    b = np.zeros(n)
    b[-1] = 1
    pi = np.linalg.solve(A, b)

    return pi

Next we will use iterative methods to determine what the steady state could be.
Through testing the best method I found was a monte-carlo esque method of just iteratively and randomly visiting nodes and keeping track of the visits.
Normalizing this gives a pretty close estimate to our steady state.
The method used is popularly known as *Monte Carlo estimation of the stationary distribution*

In [ ]:
def simulateSteadyStateVector(start=0, attempts=1_000_000):
    state = start
    visits = np.zeros(P.shape[0])
    for _ in range(attempts):
        visits[state] += 1
        probabilities = P[state]
        state = random.choices(range(len(probabilities)), weights=probabilities)[0]
    return visits / np.sum(visits)

Next we will use an iterative method to estimate the number of steps on average it takes to reach a target from its goal.
Knowing that the start and target are able to be changed we will keep the function quite general.
We will simply just check how many steps it takes to reach the target from the start over and over and take the average.
Another method that could be used is the Euler-Maruyama method but I found this to be simpler.

In [ ]:
def meanFirstPassageTime(start=0, finish=len(P)-1, attempts=10000):
    total_steps = 0
    for _ in range(attempts):
        state = start
        steps = 0
        while state != finish:
            probabilities = P[state]
            state = random.choices(range(len(probabilities)), weights=probabilities)[0]
            steps += 1
        total_steps += steps
    return total_steps / attempts


We can use a very similar method to find the probability of reaching a target from a start node without hitting another node.
This is done by just randomly changing the state and making sure its not the other, and if it's the target we up the count.


In [ ]:
def reachTargetBeforeOther(target, other, start=0, attempts=10000):
    count = 0
    for _ in range(attempts):
        state = start
        while state != target and state != other:
            probabilities = P[state]
            state = random.choices(range(len(probabilities)), weights=probabilities)[0]
        if state == target:
            count += 1
    return count / attempts


We can now put together all of our functions to answer the following questions:
1. Estimate the probability that A reaches D without hitting C.
2. Estimate the expected number of steps for A to reach F.
3. Estimate the expected number of steps for B to reach F.


In [7]:
print("Real steady state" + str(getSteadyState()))
print("Probability to hit D without htting C: " + str(reachTargetBeforeOther(3,2) * 100.0)+ "%")
print("Average steps to hit F from A: " + str(meanFirstPassageTime()))
print("Average steps to hit F from B: " + str(meanFirstPassageTime(start=1)))
print("Estimated Steadystate Vector: " + str(simulateSteadyStateVector()))
print("Difference in Steady State vector:" + str(getSteadyState() - simulateSteadyStateVector()))

Real steady state[0.08097166 0.21592443 0.29554656 0.2294197  0.11336032 0.06477733]
Probability to hit D without htting C: 5.2299999999999995%
Average steps to hit F from A: 18.1644
Average steps to hit F from B: 17.4814
Estimated Steadystate Vector: [0.081263 0.215544 0.295868 0.229801 0.112937 0.064587]
Difference in Steady State vector:[ 4.46599190e-05  2.34264507e-05 -3.62441296e-04  1.72703104e-04
  4.63238866e-05  7.53279352e-05]
